##### 1단계: LangChain 설치 및 설정

gcloud cli 인증


In [1]:
#gcloud auth application-default login

##### 2단계: Groq Qwen2.5 32B 설치 및 설정 지침  -> 대신 GCP gemini로 변경

In [2]:
import os
from langchain_google_vertexai import ChatVertexAI


# Vertex AI Gemini 기반 LLM
llm = ChatVertexAI(
    model="gemini-2.5-flash",   # 또는 "gemini-2.5-pro"
    temperature=0.0,
    
)


##### 3단계: NVIDIA bge-m3 설치 및 설정 -> 대신 vertex 임베딩

In [3]:
#pip install -qU langchain-nvidia-ai-endpoints

In [4]:

from langchain_google_vertexai import VertexAIEmbeddings

embeddings = VertexAIEmbeddings(model_name="text-embedding-004")

##### 4단계: Milvus 설치 및 설정 -> 대신 faiss

##### 5단계: RAG 챗봇 구축

In [5]:
import os
import json
import faiss

from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_core.prompts import ChatPromptTemplate


FAISS_INDEX_PATH = "citing_papers.faiss"
DATA_JSON_PATH   = "citing_papers.json"
MAP_JSON_PATH    = "citing_papers_to_model.json"


def load_vector_store(
    index_path: str = FAISS_INDEX_PATH,
    data_path: str = DATA_JSON_PATH,
    map_path: str = MAP_JSON_PATH,
):
    print(f"[INFO] Loading FAISS index '{index_path}'...")
    index = faiss.read_index(index_path)

    print(f"[INFO] Loading paper data '{data_path}' and map '{map_path}'...")
    with open(data_path, "r", encoding="utf-8") as f:
        paper_DATA = json.load(f)
    with open(map_path, "r", encoding="utf-8") as f:
        index_to_model_map = json.load(f)

    if not index_to_model_map:
        raise ValueError("index_to_model_map is empty. Run build_database.py first.")

    print("[INFO] Rebuilding LangChain docstore from FAISS map...")
    docstore_dict = {}
    index_to_docstore_id = {}

    for str_idx, model_name in index_to_model_map.items():
        int_idx = int(str_idx)
        info = paper_DATA.get(model_name)
        if info is None:
            raise KeyError(f"paper.json is missing entry for '{model_name}'.")

        doc_id = model_name
        text = f"model: {model_name}. " + " ".join(
            f"{key}: {value}" for key, value in info.items() if value is not None
        )

        metadata = {"model_name": model_name, **info}
        doc = Document(page_content=text, metadata=metadata)

        docstore_dict[doc_id] = doc
        index_to_docstore_id[int_idx] = doc_id

    if index.ntotal != len(index_to_docstore_id):
        print(f"[WARN] Index size ({index.ntotal}) != mapping count ({len(index_to_docstore_id)}).")

    docstore = InMemoryDocstore(docstore_dict)

    print("[INFO] Building LangChain FAISS VectorStore...")
    return FAISS(
        embedding_function=embeddings,
        index=index,
        docstore=docstore,
        index_to_docstore_id=index_to_docstore_id,
    )


vector_store = None  

try:
    vector_store = load_vector_store()
    print("[INFO] FAISS index loaded and reconstructed.")

except FileNotFoundError as e:
    print(f"[ERROR] Missing required file: {e}")
    print("Check paper.faiss, paper.json, paper_to_model.json paths.")
except Exception as e:
    import traceback
    print(f"[ERROR] Exception while loading FAISS: {e}")
    traceback.print_exc()


[INFO] Loading FAISS index 'citing_papers.faiss'...
[INFO] Loading paper data 'citing_papers.json' and map 'citing_papers_to_model.json'...
[INFO] Rebuilding LangChain docstore from FAISS map...
[INFO] Building LangChain FAISS VectorStore...
[INFO] FAISS index loaded and reconstructed.


프롬프트 정의

In [6]:


print("[INFO] RAG 프롬프트 정의 중...")

key_descriptions = key_descriptions = """
[데이터 필드 설명]

- title: 논문의 정식 제목.

- authors: 논문 저자 목록. 연구 기여자들의 전체 이름을 배열 형태로 제공한다.

- arxiv_id: ArXiv에 게시된 논문의 식별자. 버전 정보와 독립적으로 논문을 특정하는 데 사용된다.

- venue: 논문이 발표된 학회 또는 저널명. 비어 있을 경우 정식 출판 이전(arXiv 사전 공개 등) 상태를 의미한다.

- year: 논문의 출판 연도 혹은 ArXiv 게시 연도 (최초 공개 시점)

- publicationTypes: 출판 유형 정보(예: 저널, 학회, preprint 등). null인 경우 정식 분류가 제공되지 않은 상태를 의미한다.

- citationCount: 논문이 다른 연구로부터 인용된 횟수. 연구 영향력 및 확산 정도를 나타내는 지표이다.

- fieldsOfStudy: 논문이 속하는 학문 분야 목록. 연구의 범위 및 적용 영역을 설명한다.

- abstract: 논문의 공식 초록. 연구의 동기, 목적, 방법론, 주요 결과, 결론을 요약한 핵심 텍스트이다.

- abstract_summary_gcp: abstract를 기반으로 생성된 요약 정보(Google Cloud Pipeline 등 자동 요약).

- keywords: 논문의 주요 개념, 연구 주제, 핵심 토픽을 요약한 키워드 목록. 

- url: Semantic Scholar 등에서 해당 논문을 참조할 수 있는 링크. 원문 조회를 위한 메타데이터이다.

- isOpenAccess: 논문이 오픈 액세스 형태로 제공되는지 여부. True이면 누구나 접근 가능함을 의미한다.
"""

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 논문/모델 정보를 설명하는 어시스턴트입니다.\n"
            f"제공되는 정보는 다음과 같은 구조로 이루어져 있습니다:\n{key_descriptions}\n"
            "주어진 context(논문 메타데이터와 설명)에 근거해서만 답변하세요. "
            "모르면 모른다고 말하세요."
        ),
        (
            "human",
            "질문: {question}\n\n"
            "다음은 검색된 논문/모델 정보입니다:\n"
            "{context}\n\n"
            "위의 데이터 필드 설명과 정보를 바탕으로 한국어로 자세히 답변해 주세요."
        ),
    ]
)

print("[INFO] 프롬프트 설정 완료")


[INFO] RAG 프롬프트 정의 중...
[INFO] 프롬프트 설정 완료


In [7]:

from datetime import datetime

TOP_TIER_VENUES = {"NEURIPS", "ICML", "ICLR", "CVPR", "ACL", "NAACL", "EMNLP", "ECCV"}
DEFAULT_YEAR_WINDOW = 5
RECENCY_WEIGHT = 0.1 #최신 논문 가중치 
VENUE_WEIGHT = 0.3 #탑티어 학회 점수 가중치


# 사용자의 선호 연도와 선호 학회를 입력 받음.
def collect_user_preferences():
    print("[SETUP] Enter your search preferences.")
    #최근 몇 년 논문에 집중할지? (숫자 입력, 기본값 5)
    year_raw = input("최근 몇 년 논문을 우선할까요? (숫자 입력, 기본값 5):: ").strip()
    try:
        year_window = int(year_raw) if year_raw else DEFAULT_YEAR_WINDOW
    except ValueError:
        print("[경고] 숫자가 아닙니다. 기본값 5년을 사용합니다.")
        year_window = DEFAULT_YEAR_WINDOW

    #학회 우선순위를 어떻게 둘지?
    venue_raw = input("학회 우선순위를 선택하세요. (top/top-only/none or comma list, e.g., NeurIPS,ICML): ").strip()

    venue_mode = "all"
    preferred = []
    if venue_raw:
        lower = venue_raw.lower()
        if lower in ("top", "top-tier", "top priority"): # 탑티어에 점수 부여 + 다른 학회도 봄 
            venue_mode = "top"
        elif lower in ("top-only", "top tier only", "top-tier only"): # 탑티어만 
            venue_mode = "top_only"
        else:
            venue_mode = "custom" # 직접 지정
            preferred = [v.strip().upper() for v in venue_raw.split(",") if v.strip()]

    return {
        "year_window": year_window, # 최근 몇 년 논문에 집중할지(default 5년)
        "venue_mode": venue_mode, # top-tier 우선, top-tier only, custom venue, 전체 
        "preferred_venues": preferred, # 사용자가 직접 지정한 venue 리스트
    }


#사용자의 선호 설정 문자열로 요약 (dict -> str)
def describe_preferences(preferences: dict) -> str:
    parts = [f"recent {preferences.get('year_window', DEFAULT_YEAR_WINDOW)} years"]
    venue_mode = preferences.get("venue_mode", "all")
    if venue_mode == "top":
        parts.append("top-tier prioritized")
    elif venue_mode == "top_only":
        parts.append("top-tier only")
    elif venue_mode == "custom":
        prefs = preferences.get("preferred_venues", [])
        if prefs:
            parts.append("venues: " + ", ".join(prefs))
    else:
        parts.append("all venues")
    return " | ".join(parts)


def rerank_with_preferences(docs_with_scores, preferences: dict):
    # 검색 결과(docs_with_scores)에 선호도 기반 가중치를 더해 재정렬.
    preferences = preferences or {}
    year_window = preferences.get("year_window", DEFAULT_YEAR_WINDOW)
    venue_mode = preferences.get("venue_mode", "all")
    preferred = {v.upper() for v in preferences.get("preferred_venues", [])}
    current_year = datetime.now().year

    applied = {"year_cutoff": None, "venue_mode": venue_mode}
    cutoff = current_year - year_window if year_window else None

    filtered = docs_with_scores
    
    #연도 필터링 
    if cutoff is not None:
        #필터링 후 결과가 비면 필터링을 포기하고 원래 목록을 그대로 사용
        filtered = [ 
            (doc, score)
            for doc, score in filtered
            if int(doc.metadata.get("year", 0) or 0) >= cutoff
        ]
        applied["year_cutoff"] = cutoff
        if not filtered: 
            filtered = docs_with_scores
            applied["year_cutoff"] = None

    reranked = []
    for doc, score in filtered:
        meta = doc.metadata or {}
        #선호도 기반 점수 재계산
        base_similarity = -float(score) 
        #최신 연도 가중치
        year = meta.get("year")
        recency_bonus = 0.0
        if isinstance(year, (int, float)):
            years_old = max(current_year - int(year), 0)
            if year_window:
                recency_bonus = max(0, year_window - years_old) * RECENCY_WEIGHT
        #학회 가중치
        venue = str(meta.get("venue", "")).upper()
        venue_bonus = 0.0
        if venue_mode == "top":
            if venue in TOP_TIER_VENUES:
                venue_bonus = VENUE_WEIGHT
        elif venue_mode == "top_only":
            if venue in TOP_TIER_VENUES:
                venue_bonus = VENUE_WEIGHT
            else:
                continue # top-tier 아니면 아예 제외
        elif venue_mode == "custom":
            if venue in preferred:
                venue_bonus = VENUE_WEIGHT
        #최종 점수
        final_score = base_similarity + recency_bonus + venue_bonus
        reranked.append((doc, final_score))
    #점수가 높은 순으로 재정렬.
    if not reranked:
        reranked = [(doc, -float(score)) for doc, score in docs_with_scores]

    reranked.sort(key=lambda x: x[1], reverse=True)
    return reranked, applied


(2) RAG 한 번 호출하는 함수만 두기

In [8]:

# ===============================
# RAG 정의 
# ===============================

# Similarity filtering thresholds (L2 distance: smaller -> closer)
MAX_RESULTS = 4
RAW_K_MULTIPLIER = 3  # fetch this many before filtering
BEST_DISTANCE_MAX = 1.1  # above this, treat as unrelated
RELATIVE_MARGIN = 0.12   # allow within 12% of best distance

def filter_by_similarity(docs_with_scores, max_results: int = MAX_RESULTS):
    """Filter based on L2 distance relative to the best hit."""
    if not docs_with_scores:
        return [], {"best": None, "cutoff": None, "kept": 0}

    scores = [score for _, score in docs_with_scores]
    best = min(scores)
    cutoff = min(BEST_DISTANCE_MAX, best * (1 + RELATIVE_MARGIN))

    filtered = [
        (doc, score)
        for doc, score in docs_with_scores
        if score <= cutoff
    ]
    filtered.sort(key=lambda x: x[1])  # ascending distance (closer first)
    filtered = filtered[:max_results]
    return filtered, {"best": best, "cutoff": cutoff, "kept": len(filtered)}


def rag_answer(question: str, k: int = MAX_RESULTS, preferences: dict | None = None):
    """FAISS search + LLM answer."""
    if vector_store is None:
        raise RuntimeError("vector_store is not initialized. Run the FAISS load cell first.")

    max_results = min(k, MAX_RESULTS)
    k_search = max_results * RAW_K_MULTIPLIER
    print(f"[RAG] Query: {question}")
    docs_with_scores = vector_store.similarity_search_with_score(question, k=k_search)
    print(f"[RAG] First-pass results: {len(docs_with_scores)}")

    # similarity gate
    filtered_docs, sim_meta = filter_by_similarity(docs_with_scores, max_results=max_results)
    if not filtered_docs:
        print("[RAG] No documents passed the similarity gate.")
        return (
            "No papers sufficiently match the question. Please ask more specifically.",
            [],
            {"year_cutoff": None, "venue_mode": None, "no_results": True, **sim_meta},
        )
    print(f"[RAG] Best dist: {sim_meta['best']:.4f}, cutoff: {sim_meta['cutoff']:.4f}, kept: {sim_meta['kept']}")

    # rerank by preferences
    reranked_docs, applied_filters = rerank_with_preferences(filtered_docs, preferences or {})
    applied_filters.update(sim_meta)
    docs = [doc for doc, _ in reranked_docs]
    print(f"[RAG] After filters: {len(docs)}")

    # concatenate context
    context_text = "\n\n".join(doc.page_content for doc in docs)
    messages = prompt.invoke({"question": question, "context": context_text})
    response = llm.invoke(messages)

    return response.content, docs, applied_filters


In [13]:

print("--- Paper Chatbot ---")

if vector_store is None: #FAISS 인덱스가 로드되지 않았다면 실행할 수 없으므로 사용자에게 오류 전달
    print("[ERROR] Vector store is not loaded. Run the FAISS load cell first.")
else:
    try:
        preferences = collect_user_preferences() #사용자 선호도 입력받기
        print(f"[INFO] Preferences: {describe_preferences(preferences)}")

        user_question = input("Enter your question: ").strip()

        print(f" [You]: {user_question}")
        answer, used_docs, applied = rag_answer(user_question, k=4, preferences=preferences)

        print(f"[Bot]: {answer}")
        
        # 실제로 RAG에서 어떤 필터가 적용되었는지 사용자에게 피드백
        note = []
        if applied.get("year_cutoff"):
            note.append(f"filtered to >= {applied['year_cutoff']} year")
        vm = applied.get("venue_mode")
        if vm == "top":
            note.append("top-tier prioritized")
        elif vm == "top_only":
            note.append("top-tier only")
        elif vm == "custom":
            note.append("custom venues prioritized")
        if note:
            print("[Filters] " + " | ".join(note))
        # 사용된 문서(Sources) 출력
        print("--- Sources ---")
        for i, doc in enumerate(used_docs):
            model_name = doc.metadata.get("model_name", "Unknown")
            year = doc.metadata.get("year", "?")
            venue = doc.metadata.get("venue", "?")
            preview = doc.page_content[:120].replace("\n", " ")
            print(f"[{i+1}] {model_name} ({year}, {venue}): {preview}...")

    except Exception as e:
        import traceback
        print(f"[ERROR] RAG run failed: {e}")
        traceback.print_exc()


--- Paper Chatbot ---
[SETUP] Enter your search preferences.
[INFO] Preferences: recent 3 years | top-tier prioritized
 [You]: 게임과 관련된 논문 찾아줘
[RAG] Query: 게임과 관련된 논문 찾아줘
[RAG] First-pass results: 12
[RAG] Best dist: 1.0410, cutoff: 1.1000, kept: 4
[RAG] After filters: 4
[Bot]: 제공해주신 논문/모델 정보 중 '게임'과 직접적으로 관련된 내용은 찾을 수 없습니다.

제공된 논문들은 주로 다음과 같은 주제를 다루고 있습니다:
*   **Empathy Level Prediction in Multi-Modal Scenario with Supervisory Documentation Assistance**: 멀티모달(비디오, 오디오, 텍스트) 데이터를 활용한 공감 수준 예측 방법에 대한 연구입니다.
*   **Reasoning-Aware Multimodal Fusion for Hateful Video Detection**: 온라인 비디오에서 혐오 콘텐츠를 탐지하기 위한 멀티모달 융합 프레임워크에 대한 연구입니다.
*   **DeepSeek-V3.2: Pushing the Frontier of Open Large Language Models**: 대규모 언어 모델(LLM)의 계산 효율성, 추론 능력 및 에이전트 성능을 개선하는 새로운 모델에 대한 설명입니다.
*   **A Theoretical Framework for Auxiliary-Loss-Free Load Balancing of Sparse Mixture-of-Experts in Large-Scale AI Models**: 대규모 AI 모델에서 희소 혼합 전문가(s-MoE)의 로드 밸런싱에 대한 이론적 프레임워크를 제시합니다.
[Filters] filtered to >= 2022 year | top-t